In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import talib as ta
from itertools import combinations

import os, sys, warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [2]:
def reduce_mem_usage(df, verbose=0):
    """
    Iterate through all numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    # Calculate the initial memory usage of the DataFrame
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                # Check if the column's data type is a float
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
                    
    if verbose:
        print(f"Memory usage of dataframe is {start_mem:.2f} MB")
        end_mem = df.memory_usage().sum() / 1024**2
        print(f"Memory usage after optimization is: {end_mem:.2f} MB")
        decrease = 100 * (start_mem - end_mem) / start_mem
        print(f"Decreased by {decrease:.2f}%")

    return df

In [3]:
def gen_v1_features(df, prices):

    # V1 features: directly apply formula to a single row
    v1_features = {
        "volume": "ask_size + bid_size",
        "mid_price": "(ask_price + bid_price)/2",
        "liquidity_imbalance": "(bid_size-ask_size)/(bid_size+ask_size)",
        "matched_imbalance": "(imbalance_size - matched_size)/(matched_size+imbalance_size)",
        "size_imbalance": "bid_size / ask_size",
        "imbalance_intensity": "imbalance_size / volume",
        "matched_intensity": "matched_size / volume",
        "price_spread": "ask_price - bid_price",
        'market_urgency': 'price_spread * liquidity_imbalance',
        'depth_pressure': '(ask_size - bid_size) * (far_price - near_price)',
        'price_pressure': 'imbalance_size * (ask_price - bid_price)',
        'imbalance_with_flag': 'imbalance_size * imbalance_buy_sell_flag',
    }

    # include pair-wise price imbalances
    for c in combinations(prices, 2):
        v1_features[f"{c[0]}_{c[1]}_imbalance"] = f"({c[0]} - {c[1]}) / ({c[0]} + {c[1]})"
    
    for k, v in v1_features.items():
        df[k] = df.eval(v)
        
    v1_feature_category = {
        'minute': 'seconds_in_bucket // 60',
        'imb_buy_side': "(imbalance_buy_sell_flag == 1)",
        'imb_sell_side': "(imbalance_buy_sell_flag == -1)",
        'first_half_session': '(seconds_in_bucket <= 240)',
        'second_half_session': '(seconds_in_bucket > 240)'
    }
    
    for k, v in v1_feature_category.items():
        df[k] = df.eval(v).astype(np.int8)
        
    df = reduce_mem_usage(df, verbose=0)
        
    return df, list(v1_features.keys()), list(v1_feature_category.keys())
        
    

In [4]:
def gen_v2_features(df, v2_feat_cols):
    
    # V2 features: cross-section features
    # V2 features are generated on the groupby(['date_id', 'seconds_in_bucket'])
    # These features includes:
    # 1. statistics of V1 features (non-categorical)
    # 2. rank of V1 features for each stocks (non-categorical)
    
    group = df.groupby(['date_id', 'seconds_in_bucket'])

    v2_features_stats = ['mean', 'median', 'std', 'min', 'max']

    # calculate statistics of V1 features for each stock
    df_v2 = group[v2_feat_cols].agg(v2_features_stats).reset_index()
    df_v2.columns = ['date_id', 'seconds_in_bucket'] + [f"{c[1]}_{c[0]}" for c in df_v2.columns[2:]]
    df = df.merge(df_v2, on=['date_id', 'seconds_in_bucket'], how='left')
    

    # calculate rank of V1 features for each stock
    df_v2 = group[v2_feat_cols].rank(pct=True).add_prefix('rank_')
    df = df.merge(df_v2, left_index=True, right_index=True, how='left')
    
    df = reduce_mem_usage(df, verbose=0)
    
    v2_features =\
        [f"{s}_{c}" for c in v2_feat_cols for s in v2_features_stats] + \
        [f"rank_{c}" for c in v2_feat_cols]
        
    return df, v2_features
    

In [5]:
def gen_v3_features(df, prices, sizes, v1_features):
    # V3 features: rolling statistics of V1 features (non-categorical)
    # V3 features are generated on the groupby(['date_id', 'stock_id'])
    # here we introduce ta-lib functions to calculate TA indicators

    # V3.1 relative change of V1 features by shift(1)
    # for prices, we calculate the change in basis points (*1e4)
    # for other features, we calculate the change in percentage (*1e2)
    group_by_stock = df.groupby(['date_id', 'stock_id'])
    
    relative_price = group_by_stock[prices].pct_change(1).add_prefix('pct_')*1e4
    relative_others = group_by_stock[sizes+v1_features].pct_change(1).add_prefix('pct_')*1e2

    df = pd.concat([df, relative_price, relative_others], axis=1)
    v3_features = list(relative_price.columns) + list(relative_others.columns)
    
    # V3.2 Simple TA indicators
    # Those are simple TA indicators that use only one feature
    df_v3 = group_by_stock[prices + sizes + v1_features].rolling(5).agg(['mean', 'std', 'max', 'min']).reset_index()
    stats_cols = [f"{c[1]}_{c[0]}_5" for c in df_v3.columns[2:]]
    df_v3.columns = ['date_id', 'stock_id'] + stats_cols
    df_v3.set_index('_level_2_5', inplace=True)
    df_v3.drop(columns=['date_id', 'stock_id'], inplace=True)
    
    df = df.merge(df_v3, left_index=True, right_index=True, how='left')
    v3_features += df_v3.columns.tolist()
        
    # # V3.3 TA indicators that use multiple features
    def composite_ta(df):

        ad_osc = ta.ADOSC(df['ask_price'], df['bid_price'], df['wap'], df['volume'], fastperiod=3, slowperiod=5)
        macd, macdsignal, macdhist = ta.MACD(df['wap'], fastperiod=5, slowperiod=11, signalperiod=3)
        
        return pd.DataFrame({
            'ema': ta.EMA(df['wap'], timeperiod=5),
            'rsi': ta.RSI(df['wap'], timeperiod=5),
            'cci': ta.CCI(df['ask_price'], df['bid_price'], df['wap'], timeperiod=5),
            'mfi': ta.MFI(df['ask_price'], df['bid_price'], df['wap'], df['volume'], timeperiod=5),
            'ad_osc': ad_osc,
            'macd': macd,
            'macdsignal': macdsignal,
            'macdhist': macdhist
        })
    
    df_v3 = group_by_stock.apply(composite_ta) 
    v3_features += df_v3.columns.tolist()
    
    df_v3.reset_index(inplace=True)
    df_v3.set_index('level_2', inplace=True)
    df_v3.drop(columns=['date_id', 'stock_id'], inplace=True)
    
    df = pd.concat([df, df_v3], axis=1)
    
    return df, v3_features

In [6]:
df = pd.read_csv("/home/lishi/projects/Competition/data/train.csv")
df = df[~df['target'].isnull()] 

print(df.shape)
print(f"Trading days: {df['date_id'].nunique()}")
print(f"Stocks: {df['stock_id'].nunique()}")

df['far_price'] = df['far_price'].fillna(0)
df['near_price'] = df['near_price'].fillna(0)

df = reduce_mem_usage(df, verbose=1)

df.head()

(5237892, 17)
Trading days: 481
Stocks: 200
Memory usage of dataframe is 719.32 MB
Memory usage after optimization is: 344.67 MB
Decreased by 52.08%


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3.180603e+06,1,0.999812,13380277.00,0.0,0.0,0.999812,60651.500000,1.000026,8493.030273,1.0,-3.029704,0,0_0_0
1,1,0,0,1.666039e+05,-1,0.999896,1642214.25,0.0,0.0,0.999896,3233.040039,1.000660,20605.089844,1.0,-5.519986,0,0_0_1
2,2,0,0,3.028799e+05,-1,0.999561,1819368.00,0.0,0.0,0.999403,37956.000000,1.000298,18995.000000,1.0,-8.389950,0,0_0_2
3,3,0,0,1.191768e+07,-1,1.000171,18389746.00,0.0,0.0,0.999999,2324.899902,1.000214,479032.406250,1.0,-4.010201,0,0_0_3
4,4,0,0,4.475500e+05,-1,0.999532,17860614.00,0.0,0.0,0.999394,16485.539062,1.000016,434.100006,1.0,-7.349849,0,0_0_4


In [8]:
prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
categorical_cols = ["stock_id", "seconds_in_bucket", 'imbalance_buy_sell_flag']

feature_cols = prices + sizes

df, v1_features, v1_feature_category = gen_v1_features(df, prices)
feature_cols += v1_features
categorical_cols += v1_feature_category

df, v2_features = gen_v2_features(df, prices+sizes+v1_features)
feature_cols += v2_features

df, v3_features = gen_v3_features(df, prices, sizes, v1_features)
feature_cols += v3_features

df.fillna(0, inplace=True)
df.replace([np.inf, -np.inf], 0, inplace=True)

df = reduce_mem_usage(df, verbose=1)

print(len(feature_cols))

Memory usage of dataframe is 12278.31 MB
Memory usage after optimization is: 9161.28 MB
Decreased by 25.39%
452


In [ ]:
# df.to_csv("/home/lishi/projects/Competition/data/train_features.csv", index=False)

In [ ]:
# df_train_feature[
#     (df_train_feature['stock_id']==20) & 
#     ((df_train_feature['seconds_in_bucket'] ==300) | (df_train_feature['seconds_in_bucket'] ==310) | (df_train_feature['seconds_in_bucket'] ==290))
#     # np.isinf(df_train_feature['pct_reference_price_far_price_imbalance'])
#     ][
#         ['stock_id', 'seconds_in_bucket', 'pct_reference_price_far_price_imbalance', 'reference_price_far_price_imbalance', 'reference_price', 'far_price']
#     ]

In [59]:
# check if there is any missing values or infinite values
# for col in df_train_feature.columns:
#     print(col, np.isinf(df_train_feature[col]).sum(), df[col].isnull().sum())


In [16]:
def standardize(df, feature_cols):
    scaler = StandardScaler()
    df[feature_cols] = scaler.fit_transform(df[feature_cols])
    return df

In [79]:
from sklearn.model_selection import KFold
import lightgbm as lgb
import joblib
from sklearn.metrics import mean_absolute_error

In [ ]:
#  categorical_cols_idx = [df.columns.get_loc(c) for c in categorical_cols]
#  categorical_cols_idx

In [86]:
# set lgb parameters

lgb_params = {
    'learning_rate': 0.009,#0.018,
    'max_depth': 10,#9,
    'n_estimators': 700,#600,
    'num_leaves': 500,#440,
    'objective': 'mae',
    'random_state': 42,
    'reg_alpha': 0.01,
    'reg_lambda': 0.01,
    'early_stopping_rounds': 50,
    'num_threads': 24
    }

# CV strategy: KFold
# split train and valid set by date_id
# Within each fold, a continuous period of n days is used as validation set
# The start date of validation set is shifted by n day for each fold
# n = total_days / n_fold

k_fold = KFold(n_splits=10, shuffle=False, random_state=None)
kf_split = k_fold.split(df['date_id'].unique())

mae_scores = []

for fold, (train_dates, valid_dates) in enumerate(kf_split):
    print(f"Fold {fold+1}")
    
    # split train and valid set
    df_train = df[df["date_id"].isin(train_dates)]
    df_valid = df[df["date_id"].isin(valid_dates)]
    
    print(f"Train : {df_train.shape}, Valid : {df_valid.shape}")
    
    df_train_feature = df_train[categorical_cols + feature_cols]
    df_train_target = df_train["target"]
    
    df_valid_feature = df_valid[categorical_cols + feature_cols]
    df_valid_target = df_valid["target"]
    
    print(f"Train feature: {df_train_feature.shape}, Train target: {df_train_target.shape}")
    print(f"Valid feature: {df_valid_feature.shape}, Valid target: {df_valid_target.shape}")
    
    # standardize the features
    df_train_feature = standardize(df_train_feature, feature_cols)
    df_valid_feature = standardize(df_valid_feature, feature_cols)
    
    # categorical_cols_idx = [df_train_feature.columns.get_loc(c) for c in categorical_cols]
    # params['categorical_feature'] = categorical_cols_idx
    
    train_data = lgb.Dataset(
        data=df_train_feature.values, 
        label=df_train_target.values, 
        feature_name=df_train_feature.columns.tolist(), 
        categorical_feature=categorical_cols
        )
    
    valid_data = lgb.Dataset(
        data=df_valid_feature.values, 
        label=df_valid_target.values, 
        feature_name=df_valid_feature.columns.tolist(),
        categorical_feature=categorical_cols
        )
    
    model = lgb.train(lgb_params, train_data, valid_sets=[train_data, valid_data])
    
    print(f"Fold {fold+1} Trainning finished.")
    
    model_file = f"/home/lishi/projects/Competition/data/lgb_model_fold_{fold+1}.pkl" 
    joblib.dump(model, model_file)
    
    y_pred_valid = model.predict(df_valid_feature.values)

    y_pred_valid = np.nan_to_num(y_pred_valid)
    # y_valid = np.nan_to_num(df_valid_target.values)
    
    mae = mean_absolute_error(df_valid_target.values, y_pred_valid)
    mae_scores.append(mae)

    print(f"Fold {fold+1} MAE: {mae}")

print(f"Overall MAE: {np.mean(mae_scores)}")
print("Done!")

Fold 1
Train : (4719383, 464), Valid : (518509, 464)
Train feature: (4719383, 460), Train target: (4719383,)
Valid feature: (518509, 460), Valid target: (518509,)
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.425412 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 112226
[LightGBM] [Info] Number of data points in the train set: 4719383, number of used features: 460
[LightGBM] [Info] Start training from score -0.060201
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Did not meet early stopping. Best iteration is:
[699]	training's l1: 6.18817	valid_1's l1: 5.18126
Fold 1 Trainning finished.
Fold 1 MAE: 5.181262397187751
Fold 2
Train : (4721662, 464), Valid : (51623

KeyboardInterrupt: 

In [ ]:
# A simple validation split: keep the last 45 days for validation

# split_day = 435
# df_train = df[df["date_id"] <= split_day]
# df_valid = df[df["date_id"] > split_day]
# print(f"Train : {df_train.shape}, Valid : {df_valid.shape}")

# df_train_feature = df_train[categorical_cols + feature_cols]
# df_train_target = df_train["target"]

# df_valid_feature = df_valid[categorical_cols + feature_cols]
# df_valid_target = df_valid["target"]

# print(f"Train feature: {df_train_feature.shape}, Train target: {df_train_target.shape}")
# print(f"Valid feature: {df_valid_feature.shape}, Valid target: {df_valid_target.shape}")

In [13]:
# import packages
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [60]:
df_train_feature = standardize(df_train_feature, feature_cols)
df_valid_feature = standardize(df_valid_feature, feature_cols)

In [68]:
# define a Dataset class
class MyDataset(Dataset):
    def __init__(self, features, target):
        self.y = target
        self.X = features
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx, :], self.y[idx]
    
    
# define a neural network model of 2 layers
# first layer is a non-linear layer with m neurons
# second layer is a linear layer with n neuron

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )
    def forward(self, x):
        x = self.flatten(x)
        output = self.linear_relu_stack(x)
        return output
    
# define a function to train the model
def train(model, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        local_data, local_target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(local_data.float())
        loss = criterion(output, local_target.float())
        loss.backward()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

# define a function for validation
def validation(model, val_loader, criterion):
    model.eval()
    validation_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data, target in val_loader:
            local_data, local_target = data.to(device), target.to(device)
            output = model(local_data.float())
            validation_loss += criterion(output, local_target.float()).item()
            y_true.extend(target.tolist())
            y_pred.extend(output.tolist())
            
    validation_loss /= len(val_loader.dataset)
    print(f"\nValidation set: Average loss: {validation_loss:.6f}")
    print(f"\nValidation set: Mean Average Error: {mean_absolute_error(y_true, y_pred):.6f}\n")

# define a function to predict the target
def predict(model, pred_loader):
    model.eval()
    y_pred = []
    with torch.no_grad():
        for data, target in pred_loader:
            output = model(data.float())
            y_pred.extend(output.tolist())
            
    return y_pred

In [62]:
train_dataset = MyDataset(torch.from_numpy(df_train_feature.values), torch.from_numpy(df_train_target.values))
valid_dataset = MyDataset(torch.from_numpy(df_valid_feature.values), torch.from_numpy(df_valid_target.values))

# print number of data points and number of features
print(f"Number of data points: {train_dataset.X.shape[0]}")
print(f"Number of features: {train_dataset.X.shape[1]}")


Number of data points: 4742893
Number of features: 460


In [65]:
model = NeuralNetwork(460, 128, 1).to(device)

# train the model with train_dataset
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
val_loader = DataLoader(valid_dataset, batch_size=2048, shuffle=True)
optimizer = Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

for epoch in range(1, 3):
    train(model, train_loader, optimizer, criterion, epoch)
    validation(model, val_loader, criterion)

Train Epoch: 1 [0/4742893 (0%)]	Loss: 84.258804
Train Epoch: 1 [204800/4742893 (4%)]	Loss: 84.634796
Train Epoch: 1 [409600/4742893 (9%)]	Loss: 87.189972
Train Epoch: 1 [614400/4742893 (13%)]	Loss: 85.591690
Train Epoch: 1 [819200/4742893 (17%)]	Loss: 95.302406
Train Epoch: 1 [1024000/4742893 (22%)]	Loss: 88.084938
Train Epoch: 1 [1228800/4742893 (26%)]	Loss: 92.259865
Train Epoch: 1 [1433600/4742893 (30%)]	Loss: 92.992401
Train Epoch: 1 [1638400/4742893 (35%)]	Loss: 108.388931
Train Epoch: 1 [1843200/4742893 (39%)]	Loss: 93.323479
Train Epoch: 1 [2048000/4742893 (43%)]	Loss: 88.264053
Train Epoch: 1 [2252800/4742893 (47%)]	Loss: 90.768387
Train Epoch: 1 [2457600/4742893 (52%)]	Loss: 103.272415
Train Epoch: 1 [2662400/4742893 (56%)]	Loss: 84.189812
Train Epoch: 1 [2867200/4742893 (60%)]	Loss: 85.940292
Train Epoch: 1 [3072000/4742893 (65%)]	Loss: 86.421501
Train Epoch: 1 [3276800/4742893 (69%)]	Loss: 86.656960
Train Epoch: 1 [3481600/4742893 (73%)]	Loss: 87.373184
Train Epoch: 1 [36864